In [1]:
#loading all the packages 
import spacy
import scispacy
import pandas as pd
from spacy.matcher import Matcher
from spacy.matcher import DependencyMatcher
from spacy.tokens import Token
from spacy.tokens import Doc
from pandas import DataFrame
#run in python 3.7.4 (pip not conda) that worked best for me 


#importe WP Phenotyping functions
from categorizer_functions import *
#next line needs to be installes once and then comment out again
#pip3.7 install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz 
nlp = spacy.load("en_core_sci_sm")

In [2]:
#import Entitiy Linker  
from scispacy.linking import EntityLinker

#Add a component to the processing pipeline
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})



Your CPU supports instructions that this binary was not compiled to use: AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-l

In [3]:
#import medspacy for the visualizing components
import medspacy
from medspacy.visualization import visualize_ent


In [4]:
#Add the component to the processing pipeline
nlp.add_pipe('medspacy_context')

In [5]:
#Define our doc where we apply our matchers on
#Change is how you like, best make it SIMILAR (but no sensitive data) to phrases from the real notes
doc = nlp("Entire ileal epithelium removed. A jejunectomy was performed. Ileal resection performed. Inflammation at the ileocecal valve. Inflammation at the colon. Resection of the colon. Without resection of the ileum. Without resection of the ileocecal valve. No resection of the ileum. Inflammatory activity in the terminal ileum. Lesions in the cecum. Cobblestone appearances in the terminal ileum. Cobblestoning in cecum. Without Cobblestoning in cecum. Cobblestone sign was detected at ileocecal valve. Cobblestone sign was not detected at ileocecal valve. Terminal ileum was resected. Terminal ileum was not excised. Ileal resection was not performed. After the resection of the terminal ileum. Resection of cecum and terminal ileum.")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/spacy_legacy/layers/staticvectors_v1.py:45: DeprecationWarning: Out of bound index found. This was previously ignored when the indexing result contained no elements. In the future the index error will be raised. This error occurs either due to an empty slice, or if an array has zero elements even before indexing.
(Use `warnings.simplefilter('error')` to turn this DeprecationWarning into an error and get more details on the invalid index.)
  vectors_data = model.ops.gemm(model.ops.as_contig(V[rows]), W, trans2=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/spacy_legacy/layers/staticvectors_v1.py:45: DeprecationWarning: Out of bound index found. This was previously ignored when the indexing result contained no elements. In the future the index error will be raised. This error occurs either due to an empty slice, or if an array has zero elements even before indexing.
(Use `wa

In [6]:
#Visualizing the dependency parse
from spacy import displacy
displacy.render(doc)

In [7]:
# optional: returns a list of human-readable component names
nlp.pipe_names

['tok2vec',
 'tagger',
 'attribute_ruler',
 'lemmatizer',
 'parser',
 'ner',
 'scispacy_linker',
 'medspacy_context']

In [8]:
#Setting custom extensions for the Doc object in order to insert metadata such as Patient ID and date into the raw/intermediate debugging output file
#See rawoutput function in categorizer_functions module
# currently not functional but needs to work at the end for our output!

Doc.set_extension("metadata", default={}, force=True)
Doc.set_extension("output", default=dict(), force=True)

doc._.metadata = {'patid': 56348756, 'date': '10/09/2020'}
doc._.output = {
    'PatID': [],
    'Date': [],
    'CoarseCat': [],
    'FineCat': [],
    'MatchSpan': [],
    'Negation': [],
    #'SourceID': []
    }

In [9]:
#run the file for the custom extension, needs to be in the same folder 
%run -i custom_extensions.py


---- NEGATION DETECTION ----

By dependency:

negated: resection
negated: detected
negated: excised
negated: performed

By context:

negated: resection
negated: ileum
negated: resection
negated: ileocecal
negated: valve
negated: ileum
negated: Cobblestoning
negated: cecum
negated: detected
negated: ileocecal
negated: valve


In [10]:
# Visualizing the tokens with the medspacy component (also shows negated terms)
visualize_ent(doc)

In [11]:
#shows a short description of the tags
#can change to other  tags like "aux", "nsubj" etc.
spacy.explain("nmod")

'modifier of nominal'

In [28]:
#import our pattern also needs to be in the same folder
import l1_pattern

# change the part after l1_pattern. according to which pattern you want to test/apply
L1_resection_patterns = l1_pattern.l1_resection_pattern


In [29]:
#assigning our matcher here 
#look in the spacy documentation for more infoon how to interpret results
dmatcher = DependencyMatcher(nlp.vocab)
dmatcher.add("L1_resection", [L1_resection_patterns])
dmatches = dmatcher(doc)
print(dmatches)


[(12810985274670307352, [97, 95]), (12810985274670307352, [103, 100])]


In [31]:
# prints the anchor plus the token it matched on
match_id, token_ids = dmatches[0]
for i in range(len(token_ids)):
    print(L1_resection_patterns[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)

anchor_resection: resected
resection_subject: ileum
